In [1]:
from dotenv import load_dotenv
import os

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.tools import Tool # Tool, 즉 외부 도구를 사용하여 Reasoning + Acting 에이전트 구현 가능 
from langchain.agents import initialize_agent, AgentType
from langchain_core.runnables import Runnable, RunnableLambda

load_dotenv("../../.env")
api_key = os.getenv("OPEN_API_KEY")

In [3]:
# 1. LLM 구성 (gpt-4.1-mini-2025-04-14)
llm = ChatOpenAI(model="gpt-4.1-mini-2025-04-14", temperature=0.7, api_key=api_key)

In [ ]:
# 계산기 툴 정의 
def calculator_tool(input: str) -> str:
  try:
    return str(eval(input)) # 문자열 수식을 eval()로 계산 
  except Exception as e:
    return f"계산 오류: {str(e)}"

In [ ]:
# Tool.from_function을 통해 calculator_tool 함수를 LangChain Agent가 사용할 수 있는 도구로 변환 
calculator = Tool.from_function(
  func = calculator_tool,
  name = "Calculator",
  description = "산술 연산을 수행하는 계산기입니다. 예: 2024 - 1945" # LLM에게 힌트 제공 
)

# Agent
agent = initialize_agent(
  tools = [calculator],
  llm = llm,
  # 해당 agent는 설명 긱반 도구 선택 방식으로, Agent가 description을 기반으로 도구를 선택함 
  agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
  verbose = True,
)

C:\Users\user\AppData\Local\Temp\ipykernel_3276\2347630947.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [6]:
question = "광복절은 1945년에 있었어요. 2024년은 몇 주년인가요?"
res = agent.run(question)
print(res)

C:\Users\user\AppData\Local\Temp\ipykernel_3276\4046511254.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = agent.run(question)




> Entering new AgentExecutor chain...
Question: 광복절은 1945년에 있었어요. 2024년은 몇 주년인가요?
Thought: 2024년에서 1945년을 빼면 몇 주년인지 알 수 있다.
Action: Calculator
Action Input: 2024 - 1945
Observation: 79
Thought:Thought: 1945년부터 2024년까지 79년이 지났으므로, 2024년은 광복절 79주년이다.
Final Answer: 2024년은 광복절 79주년입니다.

> Finished chain.
2024년은 광복절 79주년입니다.
